<h1> UFO Sightings 🛸 </h1>
<h3> Exploratory Data Analysis (EDA) </h3>
<p>Data obtained from Kaggle: </p>
https://www.kaggle.com/datasets/ogunkoya/ufo-1149

Before starting, importing all the libraries needed

In [48]:
import pandas as pd

<h4>Step 1: Import data </h4>
<p> Data will be imported into a pandas dataframe. Next, get acquainted with the data available. </p>

In [49]:
df = pd.read_csv('UFO-1149.csv')

In [50]:
#looking at the first three records in the dataframe
df.head(5)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8831,-97.9411
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.3842,-98.5811
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2000,-2.9167
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783,-96.6458
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4181,-157.8036


In [51]:
#reviewing all columns, data types, and amount of values in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  float64
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80318 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  float64
 10  longitude             80332 non-null  float64
dtypes: float64(3), object(8)
memory usage: 6.7+ MB


In [52]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude'],
      dtype='object')

<hr>

<b>Quick Observations: </b>
- Need to reformat 'datetime' column to separate the time and date <br>
- Slightly over 80,000 entries with some missing data in state, country, and shape <br>
- Some entries that are foreign countries were lumped into the 'city' column <br>

<br>

<b>Questions I want to answer with this data: </b>
1. What time periods are captured in this dataset? <br>
2. Are there multiple reports from the same day? <br>
3. Do most sightings occur during the day or at night? <br>
4. What states have the most sightings? <br>
5. How long do sightings last on average? <br>

<hr>

In [53]:
#splitting datetime column into a two 'date' and 'time'
df['date'] = df['datetime'].apply(lambda x: x.split(' ')[0])

In [54]:
df['time'] = df['datetime'].apply(lambda x: x.split(' ')[1])

In [55]:
#confirming new columns were added
df.tail(3)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.2972,-122.2844,9/9/2013,22:00
80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011,-77.2656,9/9/2013,22:20
80331,9/9/2013 23:00,edmond,ok,us,cigar,1020.0,17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,9/30/2013,35.6528,-97.4778,9/9/2013,23:00


In [56]:
#also looking into all the countries recorded
df['country'].value_counts()

us    65114
ca     3000
gb     1905
au      538
de      105
Name: country, dtype: int64

In [57]:
# start looking to null values
df.loc[df['country'].isnull()]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.3842,-98.5811,10/10/1949,21:00
18,10/10/1973 23:00,bermuda nas,NaN,NaN,light,20.0,20 sec.,saw fast moving blip on the radar scope thin w...,1/11/2002,32.3642,-64.6786,10/10/1973,23:00
29,10/10/1979 22:00,saddle lake (canada),ab,NaN,triangle,270.0,4.5 or more min.,Lights far above&#44 that glance; then flee f...,1/19/2005,53.9706,-111.6899,10/10/1979,22:00
35,10/10/1982 07:00,gisborne (new zealand),NaN,NaN,disk,120.0,2min,gisborne nz 1982 wainui beach to sponge bay,1/11/2002,-38.6623,178.0176,10/10/1982,07:00
40,10/10/1986 20:00,holmes/pawling,ny,NaN,chevron,180.0,3 minutes,Football Field Sized Chevron with bright white...,10/8/2007,41.5234,-73.6468,10/10/1986,20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80238,9/9/2009 14:15,broomfield?lafayette,co,NaN,rectangle,120.0,2 min,Large&#44 rectangular object seen flying in br...,12/12/2009,39.9936,-105.0897,9/9/2009,14:15
80244,9/9/2009 20:17,lyman,me,NaN,light,600.0,10 mins,Two lights ran across the sky&#44 as bright as...,12/12/2009,43.5051,-70.6380,9/9/2009,20:17
80319,9/9/2013 20:15,clifton,nj,NaN,other,3600.0,~1hr+,Luminous line seen in New Jersey sky.,9/30/2013,40.8584,-74.1638,9/9/2013,20:15
80322,9/9/2013 21:00,aleksandrow (poland),NaN,NaN,light,15.0,15 seconds,Two points of light following one another in a...,9/30/2013,50.4658,22.8918,9/9/2013,21:00


In [58]:
#looking at all the unique values under state column
#a few more states the standard 50 states
df['state'].unique()

array(['tx', nan, 'hi', 'tn', 'ct', 'al', 'fl', 'ca', 'nc', 'ny', 'ky',
       'mi', 'ma', 'ks', 'sc', 'wa', 'ab', 'co', 'nh', 'wi', 'me', 'ga',
       'pa', 'il', 'ar', 'on', 'mo', 'oh', 'in', 'az', 'mn', 'nv', 'nf',
       'ne', 'or', 'bc', 'ia', 'va', 'id', 'nm', 'nj', 'mb', 'wv', 'ok',
       'ri', 'nb', 'vt', 'la', 'pr', 'ak', 'ms', 'ut', 'md', 'mt', 'sk',
       'wy', 'sd', 'pq', 'ns', 'qc', 'de', 'nd', 'dc', 'nt', 'sa', 'yt',
       'yk', 'pe'], dtype=object)

In [59]:
#looking at how many values of each unique category are recorded
df['state'].value_counts()

ca    9655
wa    4268
fl    4200
tx    3677
ny    3219
      ... 
nf      25
nt      20
pe      17
yt      13
yk       7
Name: state, Length: 67, dtype: int64

In [60]:
states = [ 'ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
           'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
           'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
           'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
           'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

In [61]:
#look at states that aren't true us states - ex. "yt" by creating another df
df2 = df[~df['state'].isin(states)]

In [62]:
#still showing some entries for country us
df2['country'].value_counts()

ca    3000
gb    1899
au     533
de     105
us      25
Name: country, dtype: int64

In [63]:
#see that those items belong to puerto rico which is a us territory not state
df2.loc[df['country'] == 'us']

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time
298,10/11/1998 14:05,vega baja (puerto rico),pr,us,cigar,120.0,2 minutes,I went out and saw this cigar shaped object&#4...,11/1/1998,18.4464,-66.3881,10/11/1998,14:05
7437,1/10/1999 01:00,san lorenzo (puerto rico),pr,us,formation,60.0,0:01min.,I was with my parent inside our car when sudde...,11/2/1999,18.1914,-65.9614,1/10/1999,01:00
8305,11/1/2002 23:30,arroyo (puerto rico),pr,us,triangle,30.0,30secounds,we saw what it looked to be a triangle shap ob...,11/4/2002,17.9678,-66.0617,11/1/2002,23:30
9350,11/15/1989 23:00,moca (puerto rico),pr,us,unknown,60.0,30-60 seconds,Late&#44 cloudy night in Moca&#44 Puerto Rico....,2/16/1999,18.3967,-67.1136,11/15/1989,23:00
21436,12/31/1997 21:05,puerto nuevo (puerto rico),pr,us,cross,210.0,3.5min,While visiting relatives in Puerto Rico in Dec...,3/7/1998,18.4136,-66.0822,12/31/1997,21:05
24133,1/31/1974 15:00,ponce (puerto rico),pr,us,sphere,900.0,10 to 15 minutes,The spaceship was shaped like two spheres atta...,2/16/1999,18.0131,-66.6144,1/31/1974,15:00
24141,1/31/1998 18:00,arecibo (just s of&#44 rural central pr),pr,us,disk,900.0,15 min. +,One-two hours after driving away from Arecibo ...,4/25/2002,18.4744,-66.7161,1/31/1998,18:00
32011,3/17/2001 04:00,guanica (lajas) (puerto rico),pr,us,oval,300.0,5 minutes,SIGHTING FOR MY FAMILY,8/5/2001,17.9736,-66.9083,3/17/2001,04:00
36065,4/1/1998 20:00,vega alta (puerto rico) (north side),pr,us,sphere,60.0,1 minute or 2,At around 8:00 pm on an Apirl evening NASA was...,1/28/1999,18.4142,-66.3317,4/1/1998,20:00
36827,4/15/1983 21:00,rincon (puerto rico),pr,us,circle,60.0,1 min,Two bright object&#44 one in pursuit of the other,4/22/2003,18.3422,-67.2503,4/15/1983,21:00


In [65]:
# Cleaning: something like if state in [list of us states] change country to us

In [66]:
#Visualization: create df for USA only entries for visualization